# Imports

In [27]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc, precision_score, recall_score, f1_score
import joblib
import optuna
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from tqdm import tqdm
import json

# Settings

In [3]:
# Exports
LOCAL_EXPORT_FOLDER_PATH='/content/exports'
# Exports > Manual check path 
LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/manual_check_patch'
TARGET_COLUMNS=['TARGET',]
LOCAL_EXPORT_MODELIZATION_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/modelization'
MLFLOW_EXPERIMENT_NAME = 'generic_model_experiment'
LOCAL_EXPORT_MODELIZATION_EVALUATION_RESULT_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/modelization_evaluation'

# Export > General Settings
TESTING_MODE=True
TESTING_MODE_MAX_LINES=1000
TESTING_MODE_SUB_FOLDER_NAME='testing_data'
GENERAL_CHUNK_SIZE=100000


In [3]:
def display_head_of_files(base_path, file_extension='csv', chunk_size=1000):
    """
    Parcours tous les fichiers dans le répertoire donné et affiche les premières lignes de chaque fichier CSV.

    Args:
        base_path (str): Le chemin du répertoire de base où se trouvent les fichiers.
        file_extension (str): L'extension des fichiers à traiter (par défaut 'csv').
        chunk_size (int): Taille des chunks pour lire les fichiers partiellement.

    Returns:
        None
    """
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")
                
                # Lire les premières lignes du fichier CSV
                for chunk in pd.read_csv(file_path, chunksize=chunk_size):
                    print(chunk.head())
                    break  # On ne lit qu'un seul chunk pour obtenir le head
                print("\n" + "="*80 + "\n")

# Exemple d'utilisation

In [ ]:
display_head_of_files(base_path=LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH)

# Models

In [4]:
# Liste des modèles et leurs hyperparamètres
models = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': np.logspace(-3, 3, 7),
            'solver': ['newton-cg', 'lbfgs', 'liblinear']
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4, 5],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4, 5],
            'colsample_bytree': [0.3, 0.7]
        }
    },
    'LightGBM': {
        'model': lgb.LGBMClassifier(),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 62, 127],
            'boosting_type': ['gbdt', 'dart','goss']
        }
    },
    # 'CatBoost': {
    #     'model': cb.CatBoostClassifier(verbose=0),
    #     'params': {
    #         'iterations': [100, 200, 300],
    #         'learning_rate': [0.01, 0.05, 0.1],
    #         'depth': [3, 4, 5],
    #         'l2_leaf_reg': [3, 5, 7]
    #     }
    # },
    # 'SVM': {
    #     'model': SVC(probability=True),
    #     'params': {
    #         'C': np.logspace(-3, 3, 7),
    #         'kernel': ['linear', 'rbf', 'poly'],
    #         'degree': [3, 4, 5]
    #     }
    # },
    # 'KNN': {
    #     'model': KNeighborsClassifier(),
    #     'params': {
    #         'n_neighbors': [5, 10, 20],
    #         'weights': ['uniform', 'distance'],
    #         'metric': ['euclidean', 'manhattan']
    #     }
    # },
    # 'Neural Network': {
    #     'model': MLPClassifier(max_iter=500),
    #     'params': {
    #         'hidden_layer_sizes': ['50,50', '100', '100,50'],
    #         'activation': ['tanh', 'relu'],
    #         'alpha': [0.0001, 0.001, 0.01]
    #     }
    # }
}

# Entrainment

## Hyperparameter optimization method

In [6]:
# Fonction d'optimisation des hyperparamètres avec Optuna
def objective(trial, X_train, y_train):
    classifier_name = trial.suggest_categorical('classifier', list(models.keys()))
    classifier_info = models[classifier_name]
    classifier = classifier_info['model']
    params = classifier_info['params']

    trial_params = {}
    for param, values in params.items():
        if param == 'hidden_layer_sizes':
            hidden_layer_size_str = trial.suggest_categorical(param, values)
            trial_params[param] = tuple(map(int, hidden_layer_size_str.split(',')))
        elif isinstance(values[0], int):
            trial_params[param] = trial.suggest_int(param, min(values), max(values))
        elif isinstance(values[0], float):
            trial_params[param] = trial.suggest_float(param, min(values), max(values))
        else:
            trial_params[param] = trial.suggest_categorical(param, values)
    
    classifier.set_params(**trial_params)
    
    pipeline = Pipeline(steps=[
        ('classifier', classifier)
    ])
    
    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=5, scoring='accuracy', error_score='raise')
    accuracy = score.mean()
    return accuracy

## Optuna optimize hyperameters

In [7]:
# Fonction d'optimisation des hyperparamètres avec Optuna
def objective(trial, X_train, y_train):
    classifier_name = trial.suggest_categorical('classifier', list(models.keys()))
    classifier_info = models[classifier_name]
    classifier = classifier_info['model']
    params = classifier_info['params']

    trial_params = {}
    for param, values in params.items():
        if param == 'hidden_layer_sizes':
            hidden_layer_size_str = trial.suggest_categorical(param, values)
            trial_params[param] = tuple(map(int, hidden_layer_size_str.split(',')))
        elif isinstance(values[0], int):
            trial_params[param] = trial.suggest_int(param, min(values), max(values))
        elif isinstance(values[0], float):
            trial_params[param] = trial.suggest_float(param, min(values), max(values))
        else:
            trial_params[param] = trial.suggest_categorical(param, values)
    
    classifier.set_params(**trial_params)
    
    pipeline = Pipeline(steps=[
        ('classifier', classifier)
    ])
    
    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=5, scoring='accuracy', error_score='raise')
    accuracy = score.mean()
    return accuracy

# Re-entrainment method for best increase

In [8]:
# Fonction pour ré-entraîner un modèle jusqu'à atteindre une amélioration significative
def retrain_model(best_pipeline, X_train, y_train, X_test, y_test, threshold=0.01, max_iter=10):
    previous_score = 0
    for iteration in range(max_iter):
        best_pipeline.fit(X_train, y_train)
        y_pred = best_pipeline.predict(X_test)
        current_score = accuracy_score(y_test, y_pred)
        improvement = current_score - previous_score
        if improvement < threshold:
            break
        previous_score = current_score
        print(f"Iteration {iteration + 1}, Accuracy: {current_score}, Improvement: {improvement}")
    return best_pipeline, current_score

## Principal to entrain model

In [18]:
# Fonction d'optimisation des hyperparamètres avec Optuna
def objective(trial, X_train, y_train, output_dir, best_classifier_name):
    classifier_name = trial.suggest_categorical('classifier', list(models.keys()))
    classifier_info = models[classifier_name]
    classifier = classifier_info['model']
    params = classifier_info['params']

    trial_params = {}
    for param, values in params.items():
        if isinstance(values[0], int):
            trial_params[param] = trial.suggest_int(param, min(values), max(values))
        elif isinstance(values[0], float):
            trial_params[param] = trial.suggest_float(param, min(values), max(values))
        else:
            trial_params[param] = trial.suggest_categorical(param, values)
    
    classifier.set_params(**trial_params)
    
    pipeline = Pipeline(steps=[
        ('classifier', classifier)
    ])
    
    skf = StratifiedKFold(n_splits=5)
    score = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='accuracy')
    accuracy = score.mean()
    
    # Enregistrer le modèle et les paramètres après chaque essai
    model_path = os.path.join(output_dir, f'best_{best_classifier_name}_model.pkl')
    joblib.dump(pipeline, model_path)
    
    params_path = os.path.join(output_dir, f'best_{best_classifier_name}_params.json')
    with open(params_path, 'w') as f:
        json.dump(trial_params, f)
    
    print(f"Model and parameters saved: {model_path}, {params_path}")
    
    return accuracy

# Fonction pour ré-entraîner un modèle jusqu'à atteindre une amélioration significative
def retrain_model(best_pipeline, X_train, y_train, threshold=0.01, max_iter=10):
    previous_score = 0
    for iteration in range(max_iter):
        best_pipeline.fit(X_train, y_train)
        y_pred = best_pipeline.predict(X_train)
        current_score = accuracy_score(y_train, y_pred)
        improvement = current_score - previous_score
        if improvement < threshold:
            break
        previous_score = current_score
        print(f"Iteration {iteration + 1}, Accuracy: {current_score}, Improvement: {improvement}")
    return best_pipeline, current_score

def clean_column_names(df):
    df.columns = df.columns.str.replace(r'[^\w\s]', '_', regex=True)
    return df

def train_and_evaluate_models(base_path, output_folder, target_columns, max_features=5, testing=False, chunk_size=1000, testing_sub_path_name='test'):
    all_scores = {}
    total_files = sum([len(files) for r, d, files in os.walk(base_path) if any(f.endswith('application_train.csv') for f in files)])
    pbar = tqdm(total=total_files, desc="Processing files")

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('application_train.csv'):
                file_path = os.path.join(root, file)
                test_file_path = file_path.replace('application_train.csv', 'application_test.csv')

                print(f"Processing file: {file_path}")

                # Lire les fichiers CSV par chunks
                for chunk in pd.read_csv(file_path, chunksize=chunk_size):
                    test_data = pd.read_csv(test_file_path)

                    # Nettoyer les noms de colonnes
                    chunk = clean_column_names(chunk)
                    test_data = clean_column_names(test_data)

                    for target_column in target_columns:
                        print(f"Using target column: {target_column}")

                        # Calculer les corrélations et sélectionner les meilleures caractéristiques
                        correlations = chunk.corr()[target_column].abs().sort_values(ascending=False)
                        top_features = correlations.index[1:max_features+1].tolist()

                        # Séparation des features et de la cible
                        X_train = chunk[top_features]
                        y_train = chunk[target_column]

                        X_test = test_data[top_features]
                        y_test = test_data[target_column] if target_column in test_data.columns else None

                        # Vérifier le nombre de classes dans y_train
                        if y_train.nunique() < 2:
                            print(f"Skipping optimization for {target_column} as it contains only one class in the training data.")
                            continue

                        # Déterminer le chemin de sortie
                        relative_path = os.path.relpath(root, base_path)
                        output_dir = os.path.join(output_folder, relative_path, target_column)

                        if not os.path.exists(output_dir):
                            os.makedirs(output_dir)

                        # Optimisation des hyperparamètres avec Optuna
                        study = optuna.create_study(direction='maximize')
                        study.optimize(lambda trial: objective(trial, X_train, y_train, output_dir, target_column), n_trials=100)

                        print('Number of finished trials: ', len(study.trials))
                        print('Best trial:')
                        trial = study.best_trial

                        print('  Value: ', trial.value)
                        print('  Params: ')
                        for key, value in trial.params.items():
                            print('    {}: {}'.format(key, value))

                        # Enregistrer le meilleur modèle et ses paramètres
                        best_classifier_name = trial.params['classifier']
                        best_classifier_info = models[best_classifier_name]
                        best_classifier = best_classifier_info['model']
                        best_params = {k: v for k, v in trial.params.items() if k != 'classifier'}

                        best_classifier.set_params(**best_params)

                        # Création du pipeline avec le meilleur modèle
                        best_pipeline = Pipeline(steps=[
                            ('classifier', best_classifier)
                        ])

                        # Retrain the model with the entire training dataset
                        best_pipeline, _ = retrain_model(best_pipeline, X_train, y_train)

                        # Sauvegarder le meilleur modèle et ses paramètres
                        best_model_path = os.path.join(output_dir, f'best_{best_classifier_name}_model.pkl')
                        joblib.dump(best_pipeline, best_model_path)
                        best_params_path = os.path.join(output_dir, f'best_{best_classifier_name}_params.json')
                        with open(best_params_path, 'w') as f:
                            json.dump(trial.params, f)

                        # Enregistrer les données de test
                        test_data_path = os.path.join(output_dir, 'test_data.csv')
                        test_data[top_features].to_csv(test_data_path, index=False)

                        # Enregistrement des données d'entrainement 
                        train_data_path = os.path.join(output_dir, 'train_data.csv')
                        chunk.to_csv(train_data_path, index=False)

                        if y_test is not None:
                            # Évaluation du modèle
                            y_pred = best_pipeline.predict(X_test)
                            y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]  # Only use the probability for the positive class
                            accuracy = accuracy_score(y_test, y_pred)
                            roc_auc = roc_auc_score(y_test, y_pred_proba)

                            print(f"Accuracy: {accuracy}")
                            print(f"ROC AUC: {roc_auc}")
                            print(confusion_matrix(y_test, y_pred))
                            print(classification_report(y_test, y_pred))

                            # Logging avec mlflow
                            mlflow.set_experiment('credit_scoring')
                            with mlflow.start_run():
                                mlflow.log_params(trial.params)
                                mlflow.log_metric('accuracy', accuracy)
                                mlflow.log_metric('roc_auc', roc_auc)
                                mlflow.sklearn.log_model(best_pipeline, 'model')
                                mlflow.log_artifact(file_path)
                                mlflow.log_artifact(test_file_path)
                                mlflow.log_artifact(test_data_path)

                        print(f'Model saved at {best_model_path}')
                
                pbar.update(1)

    pbar.close()

    # Visualisation des scores de validation croisée
    model_names = list(all_scores.keys())
    model_scores = [score for scores in all_scores.values() for score in scores]
    model_names_repeated = [model for model in model_names for _ in range(len(all_scores[model]))]

    plot_cross_val_scores(model_scores, model_names_repeated, output_folder)

# Fonction de visualisation pour les scores de validation croisée
def plot_cross_val_scores(model_scores, model_names, output_dir):
    plt.figure(figsize=(12, 8))
    sns.boxplot(x=model_names, y=model_scores)
    plt.xlabel('Model')
    plt.ylabel('Cross-Validation Score')
    plt.title('Model Comparison - Cross-Validation Scores')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(output_dir, 'cross_val_scores.png'))
    plt.close()

## Calling modelization method

In [ ]:
train_and_evaluate_models(
    base_path=LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH, 
    output_folder=LOCAL_EXPORT_MODELIZATION_FOLDER_PATH, 
    target_columns=TARGET_COLUMNS, 
    testing=TESTING_MODE, 
    max_features=6,
    chunk_size=GENERAL_CHUNK_SIZE, 
    testing_sub_path_name=TESTING_MODE_SUB_FOLDER_NAME)

In [17]:
test_1 = pd.read_csv("./exports/manual_check_patch/testing_data/bfill/DBSCAN/ordinal/application_train.csv")

In [18]:
test_1["TARGET"].head()

0    0
1    1
2    0
3    0
4    0
Name: TARGET, dtype: int64

# Évaluation des Modèles

After training the models, the evaluate_models function is used to evaluate the performance of each model. This function calculates several performance metrics and also computes the business score based on the given formula.

Business Score Calculation
The business score is calculated using the following formula:

Business Score
=
(
Benefit
×
PPV
)
−
(
Cost
×
False Positive Rate
)
Business Score=(Benefit×PPV)−(Cost×False Positive Rate)

Where:

Benefit: The benefit obtained from a correct prediction.
PPV (Positive Predictive Value): 
TP
TP
+
FP
TP+FP
TP
​
 , where TP is the number of true positives and FP is the number of false positives.
Cost: The cost associated with an incorrect prediction.
False Positive Rate: 
FP
Total Negatives
Total Negatives
FP
​


In [34]:
def plot_roc_curve(y_test, y_pred_proba, model_name, output_dir):
    from sklearn.metrics import RocCurveDisplay
    RocCurveDisplay.from_predictions(y_test, y_pred_proba)
    plt.title(f'ROC Curve - {model_name}')
    plt.savefig(os.path.join(output_dir, f'roc_curve_{model_name}.png'))
    plt.close()

def plot_confusion_matrix(y_test, y_pred, model_name, output_dir):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(output_dir, f'confusion_matrix_{model_name}.png'))
    plt.close()

def evaluate_models(base_path, output_folder, target_columns, chunk_size=1000, benefit=1.0, cost=1.0):
    # Créer le dossier output_folder s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print(f"Output folder: {output_folder}")
    performance_metrics = []

    # Traverse base_path and find .pkl files
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pkl'):
                print(f"Analyse of : {file}")
                print(f"In dirs : {dirs}")
                print(f"On root : {root}")
                
                model_path = os.path.join(root, file)
                model_name = file.split('_')[1] if len(file.split('_')) > 1 else "unknown_model"
                target_column = next((col for col in target_columns if col in root), None)

                if target_column:
                    print(f"Analyzing model: {model_name} for target: {target_column}")
                    try:
                        # Charger le modèle et les données de test
                        best_pipeline = joblib.load(model_path)
                        test_file_path = os.path.join(os.path.dirname(model_path), 'train_data.csv')
                        
                        if not os.path.exists(test_file_path):
                            print(f"Test file not found: {test_file_path}")
                            continue
                        
                        test_data = pd.read_csv(test_file_path, chunksize=chunk_size)
                        for chunk in test_data:
                            if target_column not in chunk.columns:
                                print(f"Target column {target_column} not in chunk columns")
                                continue

                            X_test = chunk.drop(target_column, axis=1)
                            y_test = chunk[target_column]

                            # Aligner les colonnes des données de test avec celles d'entraînement
                            try:
                                X_test = X_test.reindex(columns=best_pipeline.feature_names_in_, fill_value=0)
                            except AttributeError:
                                # Si feature_names_in_ n'est pas disponible
                                # Charger les colonnes d'entraînement sauvegardées (si disponibles)
                                pass

                            # Prédictions
                            y_pred = best_pipeline.predict(X_test)
                            y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

                            # Calcul des métriques de performance
                            accuracy = accuracy_score(y_test, y_pred)
                            precision = precision_score(y_test, y_pred)
                            recall = recall_score(y_test, y_pred)
                            f1 = f1_score(y_test, y_pred)
                            roc_auc = roc_auc_score(y_test, y_pred_proba)

                            # Matrice de confusion
                            cm = confusion_matrix(y_test, y_pred)
                            TP = cm[1, 1]
                            FP = cm[0, 1]
                            TN = cm[0, 0]
                            FN = cm[1, 0]

                            # Calcul du VPP et Taux de faux positifs
                            VPP = TP / (TP + FP) if (TP + FP) > 0 else 0
                            taux_fp = FP / (FP + TN) if (FP + TN) > 0 else 0

                            # Calcul du Score Métier
                            score_metier = (benefit * VPP) - (cost * taux_fp)

                            path_components = root.split(os.sep)[::-1]

                            performance_metrics.append({
                                'imputation_method': path_components[3],
                                'outlier_method': path_components[2],
                                'encoding_method': path_components[1],
                                'model': model_name,
                                'target': path_components[0],
                                'accuracy': accuracy,
                                'precision': precision,
                                'recall': recall,
                                'f1_score': f1,
                                'roc_auc': roc_auc,
                                'score_metier': score_metier
                            })

                            print(f"Accuracy: {accuracy}")
                            print(f"Precision: {precision}")
                            print(f"Recall: {recall}")
                            print(f"F1 Score: {f1}")
                            print(f"ROC AUC: {roc_auc}")
                            print(f"Score Métier: {score_metier}")

                            # Visualisation des résultats
                            plot_roc_curve(y_test, y_pred_proba, model_name, root)
                            plot_confusion_matrix(y_test, y_pred, model_name, root)
                    except Exception as e:
                        print(f"Error processing model {model_path}: {e}")

    if performance_metrics:
        # Convertir les métriques de performance en DataFrame
        performance_df = pd.DataFrame(performance_metrics)
        performance_df.to_csv(os.path.join(output_folder, 'model_performance_metrics.csv'), index=False)
        print("Performance metrics exported.")
        return performance_df
    else:
        print("No performance metrics to export.")
        return pd.DataFrame()

In [35]:
evaluate_models(
    base_path=LOCAL_EXPORT_MODELIZATION_FOLDER_PATH, 
    output_folder=LOCAL_EXPORT_MODELIZATION_EVALUATION_RESULT_FOLDER_PATH, 
    target_columns=["TARGET"], 
    chunk_size=GENERAL_CHUNK_SIZE, 
    benefit=1.0, 
    cost=1.0)

Output folder: /content/exports/modelization_evaluation
Analyse of : best_TARGET_model.pkl
In dirs : ['.ipynb_checkpoints']
On root : /content/exports/modelization/testing_data/mean/LOF/ordinal/TARGET
Analyzing model: TARGET for target: TARGET
Accuracy: 0.9272727272727272
Precision: 0.6666666666666666
Recall: 0.2222222222222222
F1 Score: 0.3333333333333333
ROC AUC: 0.8704620462046204
Score Métier: 0.6567656765676567
Analyse of : best_Logistic Regression_model.pkl
In dirs : ['.ipynb_checkpoints']
On root : /content/exports/modelization/testing_data/mean/LOF/ordinal/TARGET
Analyzing model: Logistic Regression for target: TARGET
Accuracy: 0.9272727272727272
Precision: 0.6666666666666666
Recall: 0.2222222222222222
F1 Score: 0.3333333333333333
ROC AUC: 0.8701870187018701
Score Métier: 0.6567656765676567
Analyse of : best_Random Forest_model.pkl
In dirs : ['.ipynb_checkpoints']
On root : /content/exports/modelization/testing_data/mean/LOF/ordinal/TARGET
Analyzing model: Random Forest for tar

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


Analyse of : best_XGBoost_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/LOF/onehot/TARGET
Analyzing model: XGBoost for target: TARGET
Accuracy: 0.9636363636363636
Precision: 1.0
Recall: 0.5555555555555556
F1 Score: 0.7142857142857143
ROC AUC: 0.9823982398239826
Score Métier: 1.0
Analyse of : best_TARGET_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/DBSCAN/ordinal/TARGET
Analyzing model: TARGET for target: TARGET
Error processing model /content/exports/modelization/testing_data/mean/DBSCAN/ordinal/TARGET/best_TARGET_model.pkl: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
Analyse of : best_Random Forest_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/DBSCAN/ordinal/TARGET
Analyzing model: Random Forest for target: TARGET
Accuracy: 0.8666666666666667
Precision: 0.75
Recall: 0.75
F1 Score: 0.75
ROC AUC: 0.931818

X has feature names, but GradientBoostingClassifier was fitted without feature names


Analyse of : best_SVM_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/DBSCAN/onehot/TARGET
Analyzing model: SVM for target: TARGET
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0
Score Métier: 1.0
Analyse of : best_TARGET_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/Isolation Forest/ordinal/TARGET
Analyzing model: TARGET for target: TARGET
Accuracy: 0.9039145907473309
Precision: 0.1111111111111111
Recall: 0.15384615384615385
F1 Score: 0.12903225806451615
ROC AUC: 0.5314293915040184
Score Métier: 0.05140961857379767
Analyse of : best_Logistic Regression_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/mean/Isolation Forest/ordinal/TARGET
Analyzing model: Logistic Regression for target: TARGET
Accuracy: 0.9537366548042705
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC: 0.7172789896670494
Score Métier: 0.0


...

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


Analyse of : best_XGBoost_model.pkl
In dirs : []
On root : /content/exports/modelization/testing_data/constant_0/Isolation Forest/onehot/TARGET
Analyzing model: XGBoost for target: TARGET
Accuracy: 0.9568345323741008
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC: 0.7396616541353385
Score Métier: 0.0


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


Performance metrics exported.


,imputation_method,outlier_method,encoding_method,model,target,accuracy,precision,recall,f1_score,roc_auc,score_metier
0,mean,LOF,ordinal,TARGET,TARGET,0.927273,0.666667,0.222222,0.333333,0.870462,0.656766
1,mean,LOF,ordinal,Logistic Regression,TARGET,0.927273,0.666667,0.222222,0.333333,0.870187,0.656766
2,mean,LOF,ordinal,Random Forest,TARGET,0.931818,1.000000,0.166667,0.285714,0.976898,1.000000
3,mean,LOF,label,TARGET,TARGET,0.927273,0.666667,0.222222,0.333333,0.870187,0.656766
4,mean,LOF,label,Logistic Regression,TARGET,0.927273,0.666667,0.222222,0.333333,0.869912,0.656766
...,...,...,...,...,...,...,...,...,...,...,...
183,constant_0,Isolation Forest,label,TARGET,TARGET,0.956835,0.000000,0.000000,0.000000,0.859336,0.000000
184,constant_0,Isolation Forest,label,Random Forest,TARGET,0.956835,0.000000,0.000000,0.000000,0.918860,0.000000
185,constant_0,Isolation Forest,label,XGBoost,TARGET,0.956835,0.000000,0.000000,0.000000,0.782581,0.000000
186,constant_0,Isolation Forest,onehot,Random Forest,TARGET,0.956835,0.000000,0.000000,0.000000,0.746084,0.000000


# Sélection du Modèle Final et Hyperparameter Tuning

After evaluating the models, the best performing model can be selected and further tuned using hyperparameter optimization.

In [ ]:
def select_and_tune_best_model(performance_df, base_path, output_folder, target_column):
    best_model_row = performance_df.loc[performance_df['roc_auc'].idxmax()]
    best_model_name = best_model_row['model']
    print(f"Best model: {best_model_name} with ROC AUC: {best_model_row['roc_auc']}")

    model_path = os.path.join(output_folder, best_model_name, target_column, f'best_{best_model_name}_model.pkl')
    best_pipeline = joblib.load(model_path)

    # Tuning des hyperparamètres du meilleur modèle avec Optuna
    X_train = pd.read_csv(os.path.join(base_path, 'application_train.csv')).drop(target_column, axis=1)
    y_train = pd.read_csv(os.path.join(base_path, 'application_train.csv'))[target_column]

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('  Value: ', trial.value)
    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

    best_classifier = models[best_model_name]['model']
    best_params = {k: v for k, v in trial.params.items() if k != 'classifier'}
    best_classifier.set_params(**best_params)

    best_pipeline = Pipeline(steps=[
        ('classifier', best_classifier)
    ])

    best_pipeline.fit(X_train, y_train)
    joblib.dump(best_pipeline, model_path)
    print(f"Model saved at {model_path}")

    return best_pipeline

# Interprétabilité et Explicabilité
Using SHAP to explain the predictions of the model.

In [ ]:
def explain_model(best_pipeline, X_train, output_folder):
    explainer = shap.TreeExplainer(best_pipeline.named_steps['classifier'])
    shap_values = explainer.shap_values(X_train)

    shap.summary_plot(shap_values, X_train)
    plt.savefig(os.path.join(output_folder, 'shap_summary_plot.png'))
    plt.close()

    shap.initjs()
    shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])
    plt.savefig(os.path.join(output_folder, 'shap_force_plot.png'))
    plt.close()

In [ ]:
# Summary

In [ ]:
def summarize_results(performance_df):
    best_model_row = performance_df.loc[performance_df['roc_auc'].idxmax()]
    best_model_name = best_model_row['model']
    summary = f"Best model: {best_model_name}\n"
    summary += f"Accuracy: {best_model_row['accuracy']}\n"
    summary += f"Precision: {best_model_row['precision']}\n"
    summary += f"Recall: {best_model_row['recall']}\n"
    summary += f"F1 Score: {best_model_row['f1_score']}\n"
    summary += f"ROC AUC: {best_model_row['roc_auc']}\n"
    summary += f"Business Score: {best_model_row['business_score']}\n"
    
    print(summary)

    with open(os.path.join(output_folder, 'model_summary.txt'), 'w') as f:
        f.write(summary)

    print("Summary saved.")